# Imports

In [ ]:
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch import Tensor
from torch import optim
import torch.nn as nn
import torch

import numpy as np
import unicodedata
import random
import re

In [ ]:
device = "cuda"

# Dataset

In [ ]:
spoken_to = ["Hey Jarvis, what's the weather today",
             "What time is it",
             "Where are we Jarvis",
             "Set a three minute timer",
             "Jarvis, what's the weather like today?",
             "Can you set a reminder for 3 PM, Jarvis?",
             "Hey Jarvis, play my favorite song.",
             "Jarvis, how do you define artificial intelligence?",
             "What's on my schedule for tomorrow, Jarvis?",
             "Tell me, what's the weather like today?",
             "Remind me to buy groceries at 5 PM.",
             "Play my favorite song now.",
             "Can you define artificial intelligence?",
             "What's on my schedule for tomorrow?",
             "Set a timer for 10 minutes.",
             "What's the weather forecast for tomorrow?",
             "Open Google and search for recent news.",
             "Play my favorite playlist on Spotify.",
             "Turn off the lights in the living room.",
             "Send a text message to John saying I'll be there in 10 minutes.",
             "Define the word 'serendipity.'",
             "Tell me a joke.",
             "What's the capital of France?",
             "Read me the latest email Jarvis.",
             "What's on my calendar for today?",
             "Hey Jarvis, convert 100 dollars to euros.",
             "Find a recipe for chicken Alfredo.",
             "Translate 'hello' to Spanish.",
             "What's the population of Tokyo?",
             "Remind me to buy groceries at 5 PM.",
             "Open the camera and take a photo.",
             "What's the latest stock price for Apple?",
             "Set an alarm for 7 AM tomorrow.",
             "Navigate to the nearest gas station.",
             "Hey Jarvis, set a timer for 15 minutes.",
             "Can you check my email, Jarvis?",
             "Hey, what's the weather like today?",
             "Jarvis, play some relaxing music, please.",
             "Could you remind me to call mom later, Jarvis?",
             "Hey Jarvis, turn on the lights in the living room.",
             "Can you find a good recipe for chocolate chip cookies, Jarvis?",
             "Jarvis, tell me a joke to lighten the mood.",
             "Hey, Jarvis, read me the latest news headlines.",
             "Could you add 'buy birthday gift' to my to-do list, Jarvis?",
             "Jarvis, what's on my calendar for the day?",
             "Hey Jarvis, find directions to the nearest coffee shop.",
             "Can you set an appointment for me at 2 PM, Jarvis?",
             "Hey, Jarvis, check the stock prices for my portfolio.",
             "Jarvis, translate 'hello' to French for me.",
             "Could you open the camera and take a picture, Jarvis?",
             "Hey Jarvis, what's the population of New York City?",
             "Jarvis, could you order a pizza for delivery?",
             "Can you tell me a fun fact, Jarvis?",
             "Hey, Jarvis, send a text to Sarah that I'll be there soon.",
             "What's on my calendar for tomorrow?",
             "Find directions to the nearest gas station.",
             "Set a meeting for 3 PM tomorrow.",
             "Check stock prices for my favorite companies.",
             "Translate 'thank you' to Spanish for me.",
             "Open the camera and take a quick photo.",
             "What's the population of Tokyo?",
             "Order a large pepperoni pizza for delivery.",
             "Tell me an interesting fact.",
             "Send a text to Mark that I'll be there in 10 minutes.",
             "I think our work here is done"]
not_spoken_to = ["How was your day?",
                 "I missed you!",
                 "What's new in your life?",
                 "Let's catch up soon.",
                 "I appreciate your support.",
                 "Can you keep a secret?",
                 "I love you.",
                 "How are the kids doing?",
                 "You mean a lot to me.",
                 "Remember that time we...",
                 "I need someone to talk to.",
                 "What's your favorite movie/book?",
                 "I trust you with this information.",
                 "You're always there for me.",
                 "Let's plan something together.",
                 "I value our friendship.",
                 "I'm feeling a bit down today.",
                 "Can you give me advice on...",
                 "Thanks for being understanding.",
                 "I'm here for you, no matter what.",
                 "What is for dinner",
                 "When is lunch going to be done",
                 "Can you help me move this heavy, furniture?",
                 "I need a hug right now.",
                 "Let's go grab a coffee together.",
                 "Can you pick up groceries on your way home?",
                 "What's your favorite type of cuisine?",
                 "I have a surprise for you!",
                 "Let's plan a surprise party for John.",
                 "What do you think about the latest fashion trends?",
                 "I need your opinion on this painting I'm working on.",
                 "Can you water the plants for me?",
                 "Let's go for a run together.",
                 "I have a secret, but I can't tell you.",
                 "What's your favorite childhood memory?",
                 "Can you make a decision for me?",
                 "I need relationship advice.",
                 "Let's go on a spontaneous road trip.",
                 "What's your dream job?",
                 "I have a crush on someone; what should I do?",
                 "Can you babysit my kids this weekend?",
                 "I have a feeling something big is going to happen.",
                 "Has anyone seen my phone? I can't find it.",
                 "Is someone using the TV right now? I wanted to watch something.",
                 "I left my keys on the kitchen counter. Have you seen them?",
                 "Did anyone finish the last of the milk? I was planning to use it.",
                 "I'll be in my room if anyone needs me.",
                 "Could you let me know if you're planning to use the car later?",
                 "I'm going to the store; does anyone need anything?",
                 "Who was the last one to use the computer in the study?",
                 "I'm having friends over tonight. Is that okay with everyone?",
                 "Have you seen the charger for my laptop around here?",
                 "I think I left my umbrella in the living room. Can you check?",
                 "Is it my turn to take out the trash, or did someone already do it?",
                 "I'm planning to do laundry later. Any objections or preferences?",
                 "Does anyone want the last slice of pizza in the fridge?",
                 "I'm done using the bathroom; it's all yours.",
                 "I'm taking a nap; please keep the noise down if possible.",
                 "I'll be working in the home office for the next few hours.",
                 "Has anyone seen the remote control? I can't find it anywhere.",
                 "I'm on a call; please don't interrupt unless it's urgent.",
                 "I made some coffee. Help yourselves if you want some.",
                 "Mom, when are we leaving for school",
                 "Where were you Scott"]

In [ ]:
negative_labels = torch.zeros(len(not_spoken_to))
positive_labels = torch.ones(len(spoken_to))

In [ ]:
sentences = np.concatenate((not_spoken_to, spoken_to))
labels = np.concatenate((negative_labels, positive_labels))

combined = list(zip(sentences, labels))
for i in range(5):
    np.random.shuffle(combined)
    
sentences, labels = zip(*combined)

# Process Data

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoder = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
tokenized

In [ ]:
tokenized = tokenizer.encode_plus("hello my name is nate",
                             max_length=20,
                             pad_to_max_length=True,
                             return_attention_mask=True,
                              return_tensors="pt")
with torch.no_grad():
    encodings = encoder(**tokenized)
    
last_hidden_states = encodings.last_hidden_state
bert_encodings = last_hidden_states.mean(dim=1).squeeze().numpy()

In [ ]:
last_hidden_states.shape

In [ ]:
def encodeString(text, tokenizer, encoder):
    indexed = tokenizer.encode_plus(text,
                                    max_length=20,
                                    pad_to_max_length=True,
                                    return_attention_mask=True,
                                    return_tensors="pt")
    
    with torch.no_grad():
        encodings = encoder(**indexed)
        
    last_hidden_states = encodings.last_hidden_state
    return last_hidden_states

### Store words into tokenizer

In [ ]:
class SentenceDataset(Dataset):
    def __init__(self, sentences, labels, 
                 tokenizer, encoder,
                 batch_size=16, max_length=150):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.encoder = encoder
        self.batch_size = batch_size
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def getitem(self, idx):
        encodings, labels = [], []
        start_idx = idx*self.batch_size
        for sentence, label in zip(self.sentences[start_idx:start_idx+batch_size],
                                   self.labels[start_idx:start_idx+batch_size]):
            encoding = encodeString(sentence[0], self.tokenizer, self.encoder)
            
            encodings.append(encoding)
            labels.append(label)

        encodings = torch.cat(encodings, dim=0)
        labels = torch.tensor(labels)
        return encodings, labels

In [ ]:
batch_size = 16
dataloader = SentenceDataset(inputs, labels, tokenizer, encoder, batch_size=batch_size, max_length=20)

In [ ]:
dataloader.getitem(0)[0][0].shape

In [ ]:
dataloader.getitem(0)[0].shape

# Build Model

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

    
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
class TargetClassifier(nn.Module):
    def __init__(self, hidden_size, vocab_size, emb_size, num_layers, dropout_p=0.1):
        super(TargetClassifier, self).__init__()
#         self.positional_encoding = PositionalEncoding(emb_size, dropout_p)
        self.gru_layers = nn.ModuleList([nn.GRU(emb_size if i == 0 else hidden_size,
                                       hidden_size,
                                       batch_first=True)
                                       for i in range(num_layers)])
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.layernorm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.out = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = x.to(device)
#         embedded = self.positional_encoding(x)
    
        for gru_layer in self.gru_layers:
            x, _ = gru_layer(x)
            
        x = x[:, -1, :]
        x = self.dense(x)
        x = self.dropout(F.gelu(self.layernorm(x)))
        return self.out(x)

In [ ]:
model = TargetClassifier(hidden_size=32,
                         vocab_size=len(lang.word2index),
                         emb_size=768,
                         num_layers=1,
                         dropout_p=0.1).to(device)

In [ ]:
def train_epoch(dataloader, model, optimizer, criterion, accuracy, train=True):
    global batch_size
    total_loss = 0
    total_acc = 0
    for batch in range(len(dataloader) // batch_size):
        sentences, labels = dataloader.getitem(batch)
        labels = labels.unsqueeze(1)
        
        optimizer.zero_grad()
        logits = model(sentences)
        loss = criterion(logits.to(torch.double), labels.to(torch.double).to(device))
        logits = F.sigmoid(logits)
        logits = torch.round(logits)
        acc = accuracy(logits, labels)
        
        if train:
            loss.backward()

            optimizer.step()
    
        total_loss += loss.item()
        total_acc += acc

    return total_loss / len(dataloader), total_acc / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
plot_train_losses = []
plot_val_losses = []

def train(train_dataloader, val_dataloader, model, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    global plot_train_losses
    global plot_val_losses
    global d_model
    print_train_loss_total = 0  # Reset every print_every
    plot_train_loss_total = 0  # Reset every plot_every
    print_train_acc_total = 0
    
    print_val_loss_total = 0  # Reset every print_every
    plot_val_loss_total = 0
    print_val_acc_total = 0

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.BCEWithLogitsLoss()
    accuracy = lambda y_pred, target: torch.sum(y_pred == target.to(device))
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                                optimizer, mode='min',
                                factor=0.50, patience=6)
    
    for epoch in range(1, n_epochs + 1):
        train_loss, train_acc = train_epoch(train_dataloader, model, optimizer, criterion, accuracy)
        print_train_loss_total += train_loss
        plot_train_loss_total += train_loss
        print_train_acc_total += train_acc
        
        # Evaluate validation dataloader
        val_loss, val_acc = train_epoch(val_dataloader, model, optimizer, criterion, accuracy, train=False)
        print_val_loss_total += val_loss
        plot_val_loss_total += val_loss
        print_val_acc_total += val_acc
        
        scheduler.step(val_loss)

        if epoch % print_every == 0:
            print_train_loss_avg = print_train_loss_total / print_every
            print_train_acc_avg = print_train_acc_total / print_every
            print_train_loss_total = 0
            print_train_acc_total = 0
            print_val_loss_avg = print_val_loss_total / print_every
            print_val_acc_avg = print_val_acc_total / print_every
            print_val_loss_total = 0
            print_val_acc_total = 0
            print('%s (%d %d%%) %.4f %.4f %.4f %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_train_loss_avg, print_val_loss_avg,
                                        print_train_acc_avg, print_val_acc_avg))
            print()

        if epoch % plot_every == 0:
            plot_train_loss_avg = plot_train_loss_total / plot_every
            plot_train_losses.append(plot_train_loss_avg)
            plot_train_loss_total = 0
            
            plot_val_loss_avg = plot_val_loss_total / plot_every
            plot_val_losses.append(plot_val_loss_avg)
            plot_val_loss_total = 0

    showPlot(plot_train_losses)
    showPlot(plot_val_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
len(train_dataloader.getitem(0)[0][0])

In [ ]:
batch_size = 16

train_dataloader = SentenceDataset(inputs[:90], labels[:90], tokenizer, encoder,
                                   batch_size=batch_size, max_length=20)
val_dataloader = SentenceDataset(inputs[90:], labels[90:], tokenizer, encoder,
                                 batch_size=batch_size, max_length=20)

train(train_dataloader, val_dataloader, model, 15, 
      learning_rate=1e-2, print_every=5, plot_every=5)

In [ ]:
torch.save(model, "target_recognition.pth")

In [ ]:
sentence = "humidity"
x = encodeString(sentence, tokenizer, encoder)

out = model(x)
out = F.sigmoid(out)
if torch.round(out) == 1:
    print("Addressing Jarvis")
else:
    print("Not addressing Jarvis")